<a href="https://colab.research.google.com/github/Joycechidi/Deep-Learning-/blob/master/Securing_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federatel Learning With Trusted Aggregator



**Step 1: Create Data Owners**

In [0]:
!pip install syft


import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 71kB 24.4MB/s 
     |████████████████████████████████| 1.4MB 46.6MB/s 
     |████████████████████████████████| 204kB 53.0MB/s 
     |████████████████████████████████| 389kB 48.5MB/s 
     |████████████████████████████████| 256kB 49.1MB/s 
     |████████████████████████████████| 450kB 51.2MB/s 
     |████████████████████████████████| 51kB 19.5MB/s 
     |████████████████████████████████| 276kB 46.3MB/s 
     |████████████████████████████████| 122kB 42.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

W0701 13:39:15.529377 140142607644544 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0701 13:39:15.547694 140142607644544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
%config IPCompleter.greedy=True

In [0]:
#Create a few workers

chidi = sy.VirtualWorker(hook, id="chidi")
ify = sy.VirtualWorker(hook, id="ify")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
chidi.add_workers([ify, secure_worker])
ify.add_workers([ify, secure_worker])
secure_worker.add_workers([ify, chidi])

W0701 13:39:26.984366 140142607644544 base.py:628] Worker ify already exists. Replacing old worker which could cause                     unexpected behavior
W0701 13:39:26.989235 140142607644544 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0701 13:39:26.991826 140142607644544 base.py:628] Worker ify already exists. Replacing old worker which could cause                     unexpected behavior
W0701 13:39:26.993118 140142607644544 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0701 13:39:26.995720 140142607644544 base.py:628] Worker ify already exists. Replacing old worker which could cause                     unexpected behavior
W0701 13:39:26.996656 140142607644544 base.py:628] Worker chidi already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
#A Toy Dataset
data = th.tensor([[0,0], [0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0], [0],[1],[1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by
#sending some training data to chidi and ify
chidis_data = data[0:2].send(chidi)
chidis_target = target[0:2].send(chidi)

In [0]:
ifys_data = data[2:].send(ify)
ifys_target = target[2:].send(ify)

## **Step 2: Create My Model**

In [0]:
#Initialize A Toy Model
model = nn.Linear(2,1)

**Step 3: Send a Copy of The Model to Ify and Chidi**

In [0]:
chidi_model = model.copy().send(chidi)
ify_model = model.copy().send(ify)
    
chidi_opt = optim.SGD(params=chidi_model.parameters(), lr=0.1)
ify_opt = optim.SGD(params=ify_model.parameters(), lr=0.1)

**Step 4: Train Chidi's and Ify's Models (in parallel)**

In [0]:
for i in range(10):
    
    # Train Chidi's Model
    chidi_opt.zero_grad()
    chidis_pred = chidi_model(chidis_data)
    chidis_loss = ((chidis_pred - chidis_target)**2).sum()
    chidis_loss.backward()

    chidi_opt.step()
    chidis_loss = chidis_loss.get().data

    # Train Ify's Model
    ify_opt.zero_grad()
    ifys_pred = ify_model(ifys_data)
    ifys_loss = ((ifys_pred - ifys_target)**2).sum()
    ifys_loss.backward()

    ify_opt.step()
    ifys_loss = ifys_loss.get().data
    ifys_loss
        


## **Step 5: Send Both Updated Models to a Secure Worker**

In [0]:
ify_model.move(secure_worker)
chidi_model.move(secure_worker)

## **Step 6: Average The Models**

In [0]:
with th.no_grad():
    model.weight.set_(((ify_model.weight.data + chidi_model.weight.data) / 2).get())
    model.bias.set_(((ify_model.bias.data + chidi_model.bias.data) / 2).get())

    print("Chidi:" + str(chidis_loss) + " Ify:" + str(ifys_loss))


Chidi:tensor(0.0083) Ify:tensor(0.0048)


## **Step 7: Rinse and Repeat**

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    chidi_model = model.copy().send(chidi)
    ify_model = model.copy().send(ify)

    chidi_opt = optim.SGD(params=chidi_model.parameters(), lr=0.1)
    ify_opt = optim.SGD(params=ify_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        chidi_opt.zero_grad()
        chidis_pred = chidi_model(chidis_data)
        chidis_loss = ((chidis_pred - chidis_target) ** 2).sum()
        chidis_loss.backward()

        chidi_opt.step()
        chidis_loss = chidis_loss.get().data

        # Train Alice's Model
        ify_opt.zero_grad()
        ifys_pred = ify_model(ifys_data)
        ifys_loss = ((ifys_pred - ifys_target) ** 2).sum()
        ifys_loss.backward()

        ify_opt.step()
        ifys_loss = ifys_loss.get().data

    ify_model.move(secure_worker)
    chidi_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((ify_model.weight.data + chidi_model.weight.data) / 2).get())
        model.bias.set_(((ify_model.bias.data + chidi_model.bias.data) / 2).get())
    
    print("Chidi:" + str(chidis_loss) + " Alice:" + str(ifys_loss))

Chidi:tensor(0.0089) Alice:tensor(0.0043)
Chidi:tensor(0.0039) Alice:tensor(2.0317e-05)
Chidi:tensor(0.0029) Alice:tensor(1.2747e-05)
Chidi:tensor(0.0022) Alice:tensor(2.0675e-05)
Chidi:tensor(0.0016) Alice:tensor(1.9907e-05)
Chidi:tensor(0.0012) Alice:tensor(1.7353e-05)
Chidi:tensor(0.0010) Alice:tensor(1.4533e-05)
Chidi:tensor(0.0007) Alice:tensor(1.1882e-05)
Chidi:tensor(0.0006) Alice:tensor(9.5567e-06)
Chidi:tensor(0.0004) Alice:tensor(7.5969e-06)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [0]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0575],
        [0.0468],
        [0.9412],
        [0.9304]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0138)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive.

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
chidi_x_share = 4 
ify_x_share = 7

decrypted_x = chidi_x_share + ify_x_share
decrypted_x

11

Note that neither Chidi nor Ify know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted).

The truly amazing thing, however, is that Ify and Chidi can still compute using this value! They can perform arithmetic over the hidden value! Let's say Chidi and Ify wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
chidi_x_share = 4 * 2
ify_x_share = 7 * 2

decrypted_x = chidi_x_share + ify_x_share
decrypted_x

22

In [0]:
# encrypted "11"
chidi_x_share = 4
ify_x_share = 7

# encrypted "13"
chidi_y_share = 5
ify_y_share = 8

# encrypted 11 + 13
chidi_z_share = chidi_x_share + chidi_y_share
ify_z_share = ify_x_share + ify_y_share

decrypted_z = chidi_z_share + ify_z_share
decrypted_z

24

As you can see, we just added two numbers together while they were still encrypted!!!
One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. 

Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.
This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [0]:
x = 5

Q = 23740629843760239486723

chidi_x_share = 23552870267 # <- a random number
ify_x_share = Q - chidi_x_share + x
ify_x_share

23740629843736686616461

In [0]:
(chidi_x_share + ify_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

## **Project: Build Methods for Encrpt, Decrypt, and Add**

Write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2, 5, 7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0, Q))
        
    shares.append(Q -(sum(shares) % Q) + x)
    
    return tuple(shares)


def decrypt(shares):
    return sum(shares) % Q

In [0]:
shares = encrypt(3)
shares

(12774262275557864140754, 7955286641132276664331, 3011080927070098681641)

In [0]:
decrypt(shares)

3

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(5)
y = encrypt(7)
z = add(x,y)
decrypt(z)

12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. 

However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(0.5)

5000

In [0]:
decode(23740629843760239486723)

0.0

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
chidi = chidi.clear_objects()
ify = ify.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1, 2, 3, 4, 5])

## Secret Sharing using PySyft

We can share using the simple .share() method

In [0]:
x = x.share(chidi, ify, secure_worker)

In [0]:
chidi._objects

{36114151322: tensor([2391663088107767470, 1467480730077372073, 3326523293831324093,
         3723216618766852754, 1964192121140691275])}

and as you can see, Chidi now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:48409368119 -> chidi:51424139305]
	-> (Wrapper)>[PointerTensor | me:72132040246 -> ify:92128867856]
	-> (Wrapper)>[PointerTensor | me:89651171190 -> secure_worker:30806366277]
	*crypto provider: me*

In [0]:
y.get()

tensor([ 2,  4,  6,  8, 10])

## Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [0]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = th.tensor([0.1,0.2,0.3])
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [0]:
x = th.tensor([0.1,0.2,0.3]).fix_prec().share(chidi, ify, secure_worker)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:43210789932 -> chidi:63994234335]
	-> (Wrapper)>[PointerTensor | me:53432084992 -> ify:15574064290]
	-> (Wrapper)>[PointerTensor | me:10555373489 -> secure_worker:42656734454]
	*crypto provider: me*

In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:8967195833 -> chidi:73174068248]
	-> (Wrapper)>[PointerTensor | me:10162491514 -> ify:79982910859]
	-> (Wrapper)>[PointerTensor | me:62831665848 -> secure_worker:30013613246]
	*crypto provider: me*

In [0]:
y = y.get().float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated learning with Encrypted Gradient Aggregation